# Package Management


## Basic Workflow
The simplest way to write a Julia program is to create a `.jl` file somewhere and run it using `julia`. You would usually do this with your favorite editor, but in this notebook we must do this programmatically. For example:

In [ ]:
code = """
println("Hello world")
"""

open(f->write(f, code), "my_program1.jl", "w")

Then let's run the program using a shell command:

In [ ]:
;julia my_program1.jl

If you need to use a package which is not part of the standard library, such as `PyCall`, you first need to install it using Julia's package manager `Pkg`:

In [ ]:
using Pkg
Pkg.add("PyCall")

Alternatively, in interactive mode, you can enter the `Pkg` mode by typing `]`, then type a command:

In [ ]:
]add PyCall

You can also precompile the new package to avoid the compilation delay when the package is first used:

In [ ]:
]add PyCall; precompile;

One last alternative is to use `pkg"..."` strings to run commands in your programs:

In [ ]:
pkg"add PyCall; precompile;"

Now you can import `PyCall` in any of your Julia programs:

In [ ]:
code = """
using PyCall
py"print('1 + 2 =', 1 + 2)"
"""

open(f->write(f, code), "my_program2.jl", "w")

In [ ]:
;julia my_program2.jl

You can also add packages by providing their URL (typically on github). This is useful when you want to use a package which is not in the [official Julia Package registry](https://github.com/JuliaRegistries/General), or when you want the very latest version of a package:

In [ ]:
]add https://github.com/JuliaLang/Example.jl

You can install a specific package version like this:

In [ ]:
]add PyCall@1.91.3

If you only specify version `1` or version `1.91`, Julia will get the latest version with that prefix. For example, `]add PyCall@0.91` would install the latest version `0.91.x`.

You can also update a package to its latest version:

In [ ]:
]update PyCall

You can update all packages to their latest versions:

In [ ]:
]update

If you don't want a particular package to be updated the next time you call `]update`, you can pin it:

In [ ]:
]pin PyCall

To unpin the package:

In [ ]:
]free PyCall

You can also run the tests defined in a package:

In [ ]:
]test Example

Of course, you can remove a package:

In [ ]:
]rm Example

Lastly, you can check which packages are installed using `]status` (or `]st` for short):

In [ ]:
]st

For more `Pkg` commands, type `]help`.

|Julia (in interactive mode) | Python (in a terminal)
|-----|------
|`]status` | `pip freeze`<br />or<br />`conda list`
|`]add Foo` | `pip install foo`<br />or<br />`conda install foo`
|`]add Foo@1.2` | `pip install foo==1.2`<br />or<br />`conda install foo=1.2`
|`]update Foo` | `pip install --upgrade foo`<br />or<br />`conda update foo`
|`]pin Foo` | `foo==<version>` in `requirements.txt`<br /> or<br />`foo=<version>` in `environment.yml`
|`]free Foo` | `foo` in `requirements.txt`<br />or<br />`foo` in `environment.yml`
|`]test Foo` | `python -m unittest foo`
|`]rm Foo` | `pip uninstall foo`<br />or<br />`conda remove foo`
|`]help` | `pip --help`


This workflow is fairly simple, but it means that all of your programs will be using the same version of each package. This is analog to installing packages using `pip install` without using virtual environments.


## Projects

If you want to have multiple projects, each with different libraries and library versions, you should define **projects**. These are analog to Python virtual environments.

A project is just a directory containing a `Project.toml` file and a `Manifest.toml` file:

```
my_project/
    Project.toml
    Manifest.toml
```

* `Project.toml` is similar to a `requirements.txt` file (for pip) or `environment.yml` (for conda): it lists the dependencies of the project, and compatibility constraints (e.g., `SomeDependency = 2.5`).
* `Manifest.toml` is an automatically generated file which lists the exact versions and unique IDs (UUIDs) of all the packages that Julia found, based on `Project.toml`. It includes all the implicit dependencies of the project's packages. This is useful to reproduce an environment precisely. Analog to the output of `pip --freeze`.

By default, the active project is located in `~/.julia/environments/v#.#` (where `#.#` is the Julia version you are using, such as 1.4). You can set a different project when starting Julia:

```bash
# BASH
julia --project=/path/to/my_project
```

Or you can set the `JULIA_PROJECT` environment variable:

```bash
# BASH
export JULIA_PROJECT=/path/to/my_project
julia
```

Or you can just activate a project directly in Julia (this is analog to running `source my_project/env/bin/activate` when using virtualenv):

In [ ]:
Pkg.activate("my_project")

The `my_project` directory does not exist yet, but it gets created automatically, along with the `Project.toml` and `Manifest.toml` files, when you first add a package:

In [ ]:
]add PyCall

You can also add a package via its URL:

In [ ]:
]add https://github.com/JuliaLang/Example.jl

Let's also add a package with a specific version:

In [ ]:
]add Example@0.3

Now the `Project.toml` and `Manifest.toml` files were created:

In [ ]:
;find my_project

Notice that the packages we added to the project were _not_ placed in the `my_project` directory itself. They were saved in the `~/.julia/packages` directory, the compiled files were placed in `~/.julia/compiled` director, logs were written to `~/.julia/logs` and so on.

If several projects use the same package, it will only be downloaded and built once (well, once per version). The `~/.julia/packages` directory can hold multiple versions of the same package, so it's fine if different projects use different versions of the same package. There will be no conflict, no "dependency hell".


The `Project.toml` just says that the project depends on `PyCall` and `Example`, and it specifies the UUID of this package:

In [ ]:
print(read("my_project/Project.toml", String))

UUIDs are useful to avoid name conflicts. If several people name their package `CoolStuff`, then the UUID will clarify which one we are referring to.

The `Manifest.toml` file is much longer, since it contains all the packages which `PyCall` and `Example` depend on, along with their versions (except for the standard library packages), and the dependency graph. This file should never be modified manually:


In [ ]:
print(read("my_project/Manifest.toml", String))

Note that `Manifest.toml` contains the precise version of the `Example` package that was installed, but the `Project.toml` file does not specify that version `0.3` is required. That's because Julia cannot know whether your project is supposed to work only with any version `0.3.x`, or whether it could work with other versions as well. So if you want to specify a version constraint for the `Example` package, you must add it manually in `Project.toml`. You would normally use your favorite editor to do this, but in this notebook we'll update `Project.toml` programmatically:

In [ ]:
append_config = """

[compat]
Example = "0.3"
"""

open(f->write(f, append_config), "my_project/Project.toml", "a")

Here is the updated `Project.toml` file:

In [ ]:
print(read("my_project/Project.toml", String))

Now if we try to replace `Example` 0.3 with version 0.2, we get an error:

In [ ]:
try
    pkg"add Example@0.2"
catch ex
    ex
end

Now you can run a program based on this project, and it will have the possibility to use all the packages which have been added to this project, with their specific versions. If you import a package which was not explicitly added to this project, Julia will fallback to the default project:

In [ ]:
code = """
import PyCall # found in the project
import PyPlot # not found, so falls back to default project
println("Success!")
"""

open(f->write(f, code), "my_program3.jl", "w")

In [ ]:
;julia --project=my_project my_program3.jl

## Packages
Falling back to the default project is fine, as long as you run the code on your own machine, but if you want to share your code with other people, it would be brittle to count on packages installed in _their_ default project. Instead, if you plan to share your code, you should clearly specify which packages it depends on, and use only these packages. Such a shareable project is called a **package**.

A package is a regular project (as defined above), but with a few extras:
* the `Project.toml` file must specify a `name`, a `version` and a `uuid`.
* there must be a `src/PackageName.jl` file containing a module named `PackageName`.
* you generally want to specify the `authors` and `description`, and maybe also the `license`, `repository` (e.g., the package's github URL), and some `keywords`, but all of these are optional.

It is very easy to create a new package using the `]generate` command. To define the `authors` field, `Pkg` will look up the `user.name` and `user.email` git config entries, so let's define them before we generate the package:

In [ ]:
;git config --global user.name "Alice Bob"

In [ ]:
;git config --global user.email "alice.bob@example.com"

In [ ]:
]generate MyPackages/Hello

This generated the `MyPackages/Hello/Project.toml` file (along with the enclosing directories) and the `MyPackages/Hello/src/Hello.jl` file. Let's take a look at the `Project.toml` file:

In [ ]:
print(read("MyPackages/Hello/Project.toml", String))

Notice that the project has no dependencies yet, but it has a name, a unique UUID, and a version (plus an author).

Note: if `Pkg` does not find a your name or email in the git config, it falls back to environment variables (`GIT_AUTHOR_NAME`, `GIT_COMMITTER_NAME`, `USER`, `USERNAME`, `NAME` and `GIT_AUTHOR_EMAIL`, `GIT_COMMITTER_EMAIL`, `EMAIL`).

And let's look at the `src/Hello.jl` file:

In [ ]:
print(read("MyPackages/Hello/src/Hello.jl", String))

Let's try to use the `greet()` function from the `Hello` package:

In [ ]:
try
    import Hello
    Hello.greet()
catch ex
    ex
end

Julia could not find the `Hello` package. When you're working on a package, don't forget to activate it first!

In [ ]:
]activate MyPackages/Hello

In [ ]:
import Hello
Hello.greet()

It works!

If the `Hello` package depends on other packages, we must add them:

In [ ]:
]add PyCall Example

You must not use any package which has not been added to the project. If you do, you will get a warning.

Once you are happy with your package, you can deploy it to github (or anywhere else). Then you can add it to your own projects just like any other package.

If you want to make your package available to the world via the official Julia registry, you just need to send a Pull Request to https://github.com/JuliaRegistries/General. However, it's highly recommended to automate this using the [Registrator.jl](https://github.com/JuliaRegistries/Registrator.jl) github app.

If you want to use other registries (including private registries), check out [this page](https://julialang.github.io/Pkg.jl/v1.4/registries/#).

Also check out the [`PkgTemplate`](https://github.com/invenia/PkgTemplates.jl) package, which provides more sophisticated templates for creating new packages, for example with continuous integration, code coverage tests, etc.

## Fixing Issues in a Dependency
Sometimes you may run into an issue inside one of the packages your project depends on. When this happens, you can use `Pkg`'s `dev` command to fix the issue. For example, let's pretend the `Example` package has a bug:

In [ ]:
]dev Example

This command cloned the repo into `~/.julia/dev/Example`:

In [ ]:
;ls -l "~/.julia/dev"

It also updated the `Hello` package's `Manifest.toml` file to ensure the package now uses the `Example` clone. You can see this using `]status`:

In [ ]:
]st

So you would now go ahead and edit the clone and fix the bug. Of course, you would also want to send a PR to the package's owners so the source package gets fixed. Once that happens, you can go back to the official `Example` package easily:

In [ ]:
]free Example

In [ ]:
]st

## Instantiating a Project
If you want to run someone else's project and you want to make sure you are using the exact same package versions, you can clone the project, and assuming it has a `Manifest.toml` file, you can activate the project and run `]instantiate` to install all the appropriate packages. For example, let's instantiate the `Registrator.jl` project:

In [ ]:
;git clone https://github.com/JuliaRegistries/Registrator.jl

In [ ]:
]activate Registrator.jl

In [ ]:
]instantiate

Usually, that's all you need to know about projects and packages, but let's look at bit under the hood, so you can handle less common cases.

## Load Path
When you import a package, Julia searches for it in the environments listed in the `LOAD_PATH` array. An **environment** can be a project or a directory containing a bunch of packages directly. By default, the `LOAD_PATH` array contains three elements:

In [ ]:
LOAD_PATH

Here's what these elements mean:
* `"@"` represents the active project, if any: that's the project activated via `--project`, `JULIA_PROJECT`, `]activate` or `Pkg.activate()`.
* `"@v#.#"` represents the default shared project for the version of Julia we are running. That's why it is used by default when there is no active project.
* `"@stdlib"` represents the standard library. This is not a project: it's a directory containing many packages.

If you want to see the actual paths, you can call `Base.load_path()`:

In [ ]:
Base.load_path()

You can change the load path if you want to. For example, if you want Julia to look only in the active project and in the standard library, without looking in the default project, then you can set the `JULIA_LOAD_PATH` environment variable to `"@:@stdlib"`.

If you try to run `my_program3.jl` this way, it will successfully import `PyCall`, but it will fail to import `PyPlot`, since it is not listed in `Project.toml` (however, it would successfully import any package from the standard library):

In [ ]:
try
    withenv("JULIA_LOAD_PATH"=>"@:@stdlib") do
        run(`julia --project=my_project my_program3.jl`)
    end
catch ex
    ex
end

You can also modify the `LOAD_PATH` array programmatically, for example to make all the packages in the `my_packages/` directory available to the project:

In [ ]:
push!(LOAD_PATH, "my_packages")

Now any package added to this directory will be directly available to us:

In [ ]:
]generate my_packages/Hello2

In [ ]:
using Hello2
Hello2.greet()

This is a convenience for development, as we didn't have to push this package to a repository or even add it to the project. However, it's just for development: once you're happy with your package, make sure to push it to a repo, and add it to the project normally.

## Depots
As we saw earlier, new packages you add to a project are placed in the `~/.julia/packages` directory, logs are placed in `~/.julia/logs`, and so on.

A directory like `~/.julia` which contains `Pkg` related content is called a **depot**. Julia installs all new packages in the default depot, which is the first directory in the `DEPOT_PATH` array (this array can be modified manually in Julia, or set via the `JULIA_DEPOT_PATH` environment variable):

In [ ]:
DEPOT_PATH

The default depot needs to be writeable for the current user, since that's where new packages will be written to (as well as logs and other stuff). The other depots can be read-only: they're typically used for private package registries.

You can occasionally run the `]gc` command, which will remove all unused package versions (`Pkg` will use the logs to located existing projects).

In summary: when some code runs `using Foo` or `import Foo`, the `LOAD_PATH` is used to determine _which_ specific package `Foo` refers to, while the `DEPOT_PATH` is used to determine _where_ it is. The exception is when the `LOAD_PATH` contains directories which directly contain packages: for these packages, the `DEPOT_PATH` is not used.

# Parallel Computing
Julia supports coroutines (aka green threads), multithreading (without a [GIL](https://en.wikipedia.org/wiki/Global_interpreter_lock#:~:text=A%20global%20interpreter%20lock%20(GIL,on%20a%20multi%2Dcore%20processor.) like CPython!), multiprocessing and distributed computing.

## Coroutines
Let's go back to the `fibonacci()` generator function:

In [ ]:
function fibonacci(n)
    Channel() do ch
        a, b = 1, 1
        for i in 1:n
            put!(ch, a)
            a, b = b, a + b
        end
    end
end

for f in fibonacci(10)
    println(f)
end

Under the hood, `Channel() do ... end` creates a `Channel` object, and spawns an asynchronous `Task` to execute the code in the `do ... end` block. The task is scheduled to execute immediately, but when it calls the `put!()` function on the channel to yield a value, it blocks until another task calls the `take!()` function to grab that value. You do not see the `take!()` function explicitly in this code example, since it is executed automatically in the `for` loop, in the main task. To demonstrate this, we can just call the `take!()` function 10 times to get all the items from the channel:

In [ ]:
ch = fibonacci(10)
for i in 1:10
    println(take!(ch))
end

This channel is bound to the task, therefore it is automatically closed when the task ends. So if we try to get one more element, we will get an exception:

In [ ]:
try
    take!(ch)
catch ex
    ex
end

Here is a more explicit version of the `fibonacci()` function:

In [ ]:
function fibonacci(n)
  function generator_func(ch, n)
    a, b = 1, 1
    for i in 1:n
        put!(ch, a)
        a, b = b, a + b
    end
  end
  ch = Channel()
  task = @task generator_func(ch, n) # creates a task without starting it
  bind(ch, task) # the channel will be closed when the task ends
  schedule(task) # start running the task asynchronously
  ch
end

And here is a more explicit version of the `for` loop:

In [ ]:
ch = fibonacci(10)
while isopen(ch)
  value = take!(ch)
  println(value)
end

Note that asynchronous tasks (also called "coroutines" or "green threads") are not actually run in parallel: they cooperate to alternate execution. Some functions, such as `put!()`, `take!()`, and many I/O functions, interrupt the current task's execution, at which point it lets Julia's scheduler decide which task should resume its execution. This is just like Python's coroutines.

For more details on coroutines and tasks, see [the manual](https://docs.julialang.org/en/v1/manual/control-flow/#man-tasks-1).

## Multithreading
Julia also supports multithreading. Currently, you need to specify the number of O.S. threads upon startup, by setting the `JULIA_NUM_THREADS` environment variable (or setting the `-t` argument in Julia 1.5+). In the first cell, we configured the IJulia kernel so that set environment variable is set:

In [ ]:
ENV["JULIA_NUM_THREADS"]

The actual number of threads started by Julia may be lower than that, as it is limited to the number of available cores on the machine (thanks to hyperthreading, each physical core may run two threads). Here is the number of threads that were actually started:

In [ ]:
using Base.Threads
nthreads()

Now let's run 10 tasks across these threads:

In [ ]:
@threads for i in 1:10
    println("thread #", threadid(), " is starting task #$i")
    sleep(rand()) # pretend we're actually working
    println("thread #", threadid(), " is finished")
end

Here is a multithreaded version of the `estimate_pi()` function. Each thread computes part of the sum, and the parts are added at the end:

In [ ]:
function parallel_estimate_pi(n)
    s = zeros(nthreads())
    nt = n ÷ nthreads()
    @threads for t in 1:nthreads()
        for i in (1:nt) .+ nt*(t - 1)
          @inbounds s[t] += (isodd(i) ? -1 : 1) / (2i + 1)
        end
    end
    return 4.0 * (1.0 + sum(s))
end

@btime parallel_estimate_pi(100_000_000)

The `@inbounds` macro is an optimization: it tells the Julia compiler not to add any bounds check when accessing the array. It's safe in this case since the `s` array has one element per thread, and `t` varies from `1` to `nthreads()`, so there is no risk for `s[t]` to be out of bounds.

Let's compare this with the single-threaded implementation:

In [ ]:
@btime estimate_pi(100_000_000)

If you are running this notebook on Colab, the parallel implementation is probably no faster than the single-threaded one. That's because the Colab Runtime only has a single CPU, so there is no benefit from multithreading (plus there is a bit of overhead for managing threads). However, on my 8-core machine, using 16 threads, the parallel implementation is about 6 times faster than the single-threaded one.

Julia has a `mapreduce()` function which makes it easy to implement functions like `parallel_estimate_pi()`:

In [ ]:
function parallel_estimate_pi2(n)
    4.0 * mapreduce(i -> (isodd(i) ? -1 : 1) / (2i + 1), +, 0:n)
end

In [ ]:
@btime parallel_estimate_pi2(100_000_000)

The `mapreduce()` function is well optimized, so it's about twice faster than `parallel_estimate_pi()`.

You can also spawn a task using `Threads.@spawn`. It will get executed on any one of the running threads (it will not start a new thread):

In [ ]:
task = Threads.@spawn begin
    println("Thread starting")
    sleep(1)
    println("Thread stopping")
    42 # result
end

println("Hello!")

println("The result is: ", fetch(task))


The `fetch()` function waits for the thread to finish, and fetches the result. You can also just call `wait()` if you don't need the result.

Last but not least, you can use channels to synchronize and communicate across tasks, even if they are running across separate threads:

In [ ]:
ch = Channel()
task1 = Threads.@spawn begin
    for i in 1:5
        sleep(rand())
        put!(ch, i^2)
    end
    println("Finished sending!")
    close(ch)
end

task2 = Threads.@spawn begin
    foreach(v->println("Received $v"), ch)
    println("Finished receiving!")
end

wait(task2)

For more details about multithreading, check out [this page](https://docs.julialang.org/en/v1/manual/parallel-computing/#man-multithreading-1).

## Multiprocessing & Distributed Programming
Julia can spawn multiple Julia processes upon startup if you specify the number of processes via the `-p` argument. You can also spawn extra processes from Julia itself:

In [ ]:
using Distributed
addprocs(4)
workers() # array of worker process ids

The main process has id 1:

In [ ]:
myid()

The `@everywhere` macro lets you run any code on all workers:

In [ ]:
@everywhere println("Hi! I'm worker $(myid())")

You can also execute code on a particular worker by using `@spawnat <worker id> <statement>`:

In [ ]:
@spawnat 3 println("Hi! I'm worker $(myid())")

If you specify `:any` instead of a worker id, Julia chooses the worker for you:

In [ ]:
@spawnat :any println("Hi! I'm worker $(myid())")

Both `@everywhere` and `@spawnat` return immediately. The output of `@spawnat` is a `Future` object. You can call `fetch()` on this object to wait for the result:

In [ ]:
result = @spawnat 3 1+2+3+4
fetch(result)

If you import some package in the main process, it is <u>not</u> automatically imported in the workers. For example, the following code fails because the worker does not know what `pyimport` is:

In [ ]:
using PyCall

result = @spawnat 4 (np = pyimport("numpy"); np.log(10))

try
    fetch(result)
catch ex
    ex
end

You must use `@everywhere` or `@spawnat` to import the packages you need in each worker:

In [ ]:
@everywhere using PyCall

result = @spawnat 4 (np = pyimport("numpy"); np.log(10))

fetch(result)

Similarly, if you define a function in the main process, it is <u>not</u> automatically available in the workers. You must define the function in every worker:

In [ ]:
@everywhere addtwo(n) = n + 2
result = @spawnat 4 addtwo(40)
fetch(result)

You can pass a `Future` to `@everywhere` or `@spawnat`, as long as you wrap it in a `fetch()` function:

In [ ]:
M = @spawnat 2 rand(5)
result = @spawnat 3 fetch(M) .* 10.0
fetch(result)

In this example, worker 2 creates a random array, then worker 3 fetches this array and multiplies each element by 10, then the main process fetches the result and displays it.

## GPU
Julia has excellent GPU support. As you may know, GPUs are devices which can run thousands of threads in parallel. Each thread is slower and more limited than on a CPU, but there are so many of them that plenty of tasks can be executed much faster on a GPU than on a CPU, provided these tasks can be parallelized.

Let's check which GPU device is installed:

In [ ]:
;nvidia-smi

If you're running on Colab, your runtime will generally have an Nvidia Tesla K80 GPU with 12GB of RAM installed, but sometimes other GPUs like Nvidia Tesla T4 16GB, or Nvidia Tesla P100).

If no GPU is detected, go to _Runtime_ > _Change runtime type_, set _Hardware accelerator_ to _GPU_, then go to _Runtime_ > _Factory reset runtime_, then reinstall Julia by running the first cell again, then reload the page and come back here). If you're running on your own machine, make sure you have a compatible GPU card installed, with the appropriate drivers.

Now let's create a large matrix and time how long it takes to square it on the CPU:

In [ ]:
using BenchmarkTools

M = rand(2^11, 2^11)

function benchmark_matmul_cpu(M)
    M * M
    return
end

benchmark_matmul_cpu(M) # warm up
@btime benchmark_matmul_cpu($M)

Notes:
* For benchmarking, we wrapped the operation in a function which returns `nothing`.
* Why do we have a "warm up" line? Well, since Julia compiles code on the fly the first time it is executed, it's good practice to execute the operation we want to benchmark at least once before starting the benchmark, or else the benchmark will include the compilation time.
* We used `$M` instead of `M` on the last line. This is a feature of the `@btime` macro: it evaluates `M` before benchmarking takes place, to avoid the extra delay that is incurred when [benchmarking with global variables](https://docs.julialang.org/en/latest/manual/performance-tips/#Avoid-global-variables-1).

Now let's benchmark this same operation on the GPU: 

In [ ]:
using CUDA

# Copy the data to the GPU. Creates a CuArray:
M_on_gpu = cu(M)

# Alternatively, create a new random matrix directly on the GPU:
#M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)

function benchmark_matmul_gpu(M)
    CUDA.@sync M * M
    return
end

benchmark_matmul_gpu(M_on_gpu) # warm up
@btime benchmark_matmul_gpu($M_on_gpu)

That's _much_ faster (185x faster in my test on Colab with an NVidia Tesla P100 GPU).

Importantly:
* Before the GPU can work on some data, it needs to be copied to the GPU (or generated there directly).
* the `CUDA.@sync` macro waits for the GPU operation to complete. Without it, the operation would happen in parallel on the GPU, while execution would continue on the CPU. So we would just be timing how long it takes to _start_ the operation, not how long it takes to complete.
* In general, you don't need `CUDA.@sync`, since many operations (including `cu()`) call it implicitly, and it's usually a good idea to let the CPU and GPU work in parallel. Typically, the GPU will be working on the current batch of data while the CPU works on preparing the next batch.

Of course, the speed up will vary depending on the matrix size and the GPU type. Moreover, copying the data from the CPU to the GPU is often the slowest part of the operation, but we only benchmarked the matrix multiplication itself. Let's see what we get if we include the data transfer in the benchmark:

That's still much faster than on the CPU.

Let's check how much RAM we have left on the GPU:

In [ ]:
CUDA.memory_status()

Julia's Garbage Collector will free CUDA arrays like any other object, when there's no more reference to it. However, `CUDA.jl` uses a memory pool to make allocations faster on the GPU, so don't be surprised if the allocated memory on the GPU does not go down immediately. Moreover, IJulia keeps a reference to the output of each cell, so if you let any cell output a `CuArray`, it will only be released when you execute `Out[<cell number>]=0`. If you want to force the Garbage Collector to run, you an run `GC.gc()`. To reclaim memory from the memory pool, use `CUDA.reclaim()`:

In [ ]:
GC.gc()
CUDA.reclaim()

Many other operations are implemented for `CuArray` (`+`,  `-`, etc.) and dotted operations (`.+`, `exp.()`, etc). Importantly, loop fusion also works on the GPU. For example, if we want to compute `M .* M .+ M`, without loop fusion the GPU would first compute `M .* M` and create a temporary array, then it would add `M` to that array, like this:

In [ ]:
function benchmark_without_fusion(M)
    P = M .* M
    CUDA.@sync P .+ M
    return
end

benchmark_without_fusion(M_on_gpu) # warm up
@btime benchmark_without_fusion($M_on_gpu)

Instead, loop fusion ensures that the array is only traversed once, without the need for a temporary array:

In [ ]:
function benchmark_with_fusion(M)
    CUDA.@sync M .* M .+ M
    return
end

benchmark_with_fusion(M_on_gpu) # warm up
@btime benchmark_with_fusion($M_on_gpu)

That's _much_ faster (75% faster in my test on Colab). 😃

Lastly, you can actually **write your own GPU kernels in Julia**! In other words, rather than using GPU operations implemented in the `CUDA.jl` package (or others), you can write Julia code that will be compiled for the GPU, and executed there. This can occasionally be useful to speed up some algorithms where the standard kernels don't suffice. For example, here's a GPU kernel which implements `u .+= v`, where `u` and `v` are two (large) vectors:

In [ ]:
function worker_gpu_add!(u, v)
    index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    index ≤ length(u) && (@inbounds u[index] += v[index])
    return
end

function gpu_add!(u, v)
    numblocks = ceil(Int, length(u) / 256)
    @cuda threads=256 blocks=numblocks worker_gpu_add!(u, v)
    return u
end

This code example is adapted from the [`CUDA.jl` package's documentation](https://juliagpu.gitlab.io/CUDA.jl/tutorials/introduction/), which I highly encourage you to check out if you're interested in writing your own kernels. Here are the key parts to understand this example, starting from the end:
* The `gpu_add!()` function first calculates `numblocks`, the number of blocks of threads to start, then it uses the `@cuda` macro to spawn `numblocks` blocks of GPU threads, each with 256 threads, and each thread runs `worker_gpu_add!(u, v)`.
* The `worker_gpu_add!()` function computes `u[index] += v[index]` for a single value of `index`: in other words, each thread will just update a single value in the vector! Let's see how the index is computed:
  * The `@cuda` macro spawned many blocks of 256 threads each. These blocks are organized in a grid, which is one-dimensional by default, but it can be up to three-dimensional. Therefore each thread and each block have an `(x, y, z)` coordinate in this grid. See this diagram from the [Nvidia blog post](https://developer.nvidia.com/blog/even-easier-introduction-cuda/):<br />
<img src="https://juliagpu.gitlab.io/CUDA.jl/tutorials/intro1.png" width="600"/>.
  * `threadIdx().x` returns the current GPU thread's `x` coordinate within its block (one difference with the diagram is that Julia is 1-indexed).
  * `blockIdx().x` returns the current block's `x` coordinate in the grid.
  * `blockDim().x` returns the block size along the `x` axis (in this example, it's 256).
  * `gridDim().x` returns the number of blocks in the grid, along the `x` axis (in this example it's `numblocks`).
  * So the `index` that each thread must update in the array is `(blockIdx().x - 1) * blockDim().x + threadIdx().x`.
* As explained earlier, the `@inbounds` macro is an optimization that tells Julia that the index is guaranteed to be inbounds, so there's no need for it to check.

Now writing your own GPU kernel won't seem like something only top experts with advanced C++ skills can do: you can do it too!

Let's check that the kernel works as expected:

In [ ]:
u = rand(2^20)
v = rand(2^20)

u_on_gpu = cu(u)
v_on_gpu = cu(v)

u .+= v
gpu_add!(u_on_gpu, v_on_gpu)

@assert Array(u_on_gpu) ≈ u

Yes, it works well!

Note: the `≈` operator checks whether the operands are approximately equal within the float precision limit.

Let's benchmark our custom kernel:

In [ ]:
function benchmark_custom_assign_add!(u, v)
    CUDA.@sync gpu_add!(u, v)
    return
end

benchmark_custom_assign_add!(u_on_gpu, v_on_gpu)
@btime benchmark_custom_assign_add!($u_on_gpu, $v_on_gpu)

Let's see how this compares to `CUDA.jl`'s implementation:

In [ ]:
function benchmark_assign_add!(u, v)
    CUDA.@sync u .+= v
    return
end

benchmark_assign_add!(u_on_gpu, v_on_gpu)
@btime benchmark_assign_add!($u_on_gpu, $v_on_gpu)

How about that? Our custom kernel is faster than `CUDA.jl`'s kernel! But to be fair, our kernel would not work with huge vectors, since there's a limit to the number of blocks & threads you can spawn (see [Table 15](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#features-and-technical-specifications) in CUDA's documentation). To support such huge vectors, we need each worker to run a loop like this:

In [ ]:
function worker_gpu_add!(u, v)
    index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    stride = blockDim().x * gridDim().x
    for i = index:stride:length(u)
        @inbounds u[i] += v[i]
    end
    return
end

This way, if `@cuda` is executed with a smaller number of blocks than needed to have one thread per array item, the workers will loop appropriately.

This should get you started! For more info, check out [`CUDA.jl`'s documentation](https://juliagpu.gitlab.io/CUDA.jl/).

# Command Line Arguments

Command line arguments are available via `ARGS`:



In [ ]:
ARGS

Unlike Python's `sys.argv`, the first element of this array is <u>not</u> the program name. If you need the program name, use `PROGRAM_FILE` instead:

In [ ]:
PROGRAM_FILE

You can get the current module, directory, file or line number:

In [ ]:
@__MODULE__, @__DIR__, @__FILE__, @__LINE__

The equivalent of Python's `if __name__ == "__main__"` is:

In [ ]:
if abspath(PROGRAM_FILE) == @__FILE__
    println("Starting of the program")
end

# Memory Management

Let's check how many megabytes of RAM are available:

In [ ]:
free() = println("Available RAM: ", Sys.free_memory() ÷ 10^6, " MB")

free()

If a variable holds a large object that you don't need anymore, you can either wait until the variable falls out of scope, or set it to `nothing`. Either way, the memory will only be freed when the Garbage Collector does its magic, which may not be immediate. In general, you don't have to worry about that, but if you want, you can always call the GC directly:

In [ ]:
function use_ram()
    M = rand(10000, 10000) # use 400+MB of RAM
    println("sum(M)=$(sum(M))")
end # M will be freed by the GC eventually after this

use_ram()

M = rand(10000, 10000) # use 400+MB of RAM
println("sum(M)=$(sum(M))")
M = nothing

GC.gc() # rarely needed

In [ ]:
free()

# Thanks!

I hope you enjoyed this introduction to Julia! I recommend you join the friendly and helpful Julia community on Slack or Discourse.

Cheers!

Aurélien Geron